In [190]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [191]:
folder = "/content/drive/MyDrive/Tesla/"
stocks = pd.read_csv(folder+"TSLA.csv")
stocks = stocks.rename(columns = str.lower)
stocks = stocks.rename(columns = {'adj close':'adj'})
stocks.shape

(2821, 7)

In [192]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [193]:
stocks.date = pd.to_datetime(stocks.date)
start = list(stocks.date)[0]
end = list(stocks.date)[-1]
pd.date_range(start = start, end = end).difference(stocks.date)

DatetimeIndex(['2010-07-03', '2010-07-04', '2010-07-05', '2010-07-10',
               '2010-07-11', '2010-07-17', '2010-07-18', '2010-07-24',
               '2010-07-25', '2010-07-31',
               ...
               '2021-08-08', '2021-08-14', '2021-08-15', '2021-08-21',
               '2021-08-22', '2021-08-28', '2021-08-29', '2021-09-04',
               '2021-09-05', '2021-09-06'],
              dtype='datetime64[ns]', length=1271, freq=None)

In [194]:
import datetime
len = stocks.date.shape[0]
missing = pd.DataFrame(columns=stocks.columns)
for i in range(len-1):
  dif = stocks.date[i+1] - stocks.date[i]
  if dif > datetime.timedelta(days = 1):
    x_vals = stocks.iloc[i]
    y_vals = stocks.iloc[i+1]
    for j in range(1, int(dif.days)):
      date = x_vals[0] + datetime.timedelta(days = 1)
      open = (y_vals[1] + x_vals[1])/2
      high = (y_vals[2] + x_vals[2])/2
      low = (y_vals[3] + x_vals[3])/2
      close = (y_vals[4] + x_vals[4])/2
      adj = (y_vals[5] + x_vals[5])/2
      new_row = {'date': date, 'open' : open, 'high': high, 'low' :low, 'close' : close, 'adj': adj, 'volume' :0}
      x_vals = list(new_row.values())
      missing = missing.append(new_row, ignore_index=True)
stocks = stocks.append(missing).sort_values("date").reset_index(drop = True)
stocks = stocks.iloc[:,:-1]
stocks.shape

(4092, 6)

In [212]:
folder = "/content/drive/MyDrive/Tesla/"
tweets = pd.read_csv(folder+"top2vec_tesla_stuff.csv")
tweets.date = pd.to_datetime(tweets.date).dt.date
tweets = tweets.iloc[:,2:]
tweets.Cluster = 1 #remove this
tweets.shape

(797, 19)

In [213]:
def stocks_changer(dataframe, p):
  res = dataframe.copy()
  direction = []
  for j in range(0, dataframe.shape[0]-1):
    sub = dataframe['open'][j+1] - dataframe['open'][j]
    if np.abs(sub) >= p:
      direction.append(1 if dataframe['open'][j+1] >= dataframe['open'][j] else -1)
    else:
      direction.append(0)
  direction.append(0)
  res['change'] = direction
  return res

In [214]:
check = stocks_changer(stocks, 0)
check 

,date,open,high,low,close,adj,change
0,2011-12-01,6.514000,6.798000,6.396000,6.520000,6.520000,1
1,2011-12-02,6.566000,6.738000,6.480000,6.660000,6.660000,1
2,2011-12-03,6.636000,6.869000,6.583000,6.772000,6.772000,1
3,2011-12-04,6.671000,6.934500,6.634500,6.828000,6.828000,1
4,2011-12-05,6.706000,7.000000,6.686000,6.884000,6.884000,1
...,...,...,...,...,...,...,...
3395,2021-03-18,684.289978,689.229980,652.000000,653.159973,653.159973,-1
3396,2021-03-19,646.599976,657.229980,624.619995,654.869995,654.869995,1
3397,2021-03-20,665.595001,678.424988,646.684998,662.434998,662.434998,1
3398,2021-03-21,675.092514,689.022491,657.717499,666.217499,666.217499,1


In [215]:
startdate = pd.to_datetime('2011-12-01')
enddate = pd.to_datetime('2021-03-22')
tweets = tweets.loc[(tweets.date >= startdate) & (tweets.date <= enddate)].sort_values('date').reset_index(drop = True)
stocks = stocks.loc[(stocks.date >= startdate) & (stocks.date <= enddate)].sort_values('date').reset_index(drop = True)
tweets['popularity'] = tweets.nlikes + tweets.nreplies + tweets.nretweets
tweets_daily = tweets.groupby('date',as_index = False).max(['popularity'])
tweets_daily.date = pd.to_datetime(tweets_daily.date)
tweets_daily.head()
tweets

,date,tweet,language,hashtags,cashtags,link,retweet,nlikes,nreplies,nretweets,reply_to,Cluster,subjectivity,polarity,sentiment,bertwitter_sentiment,hf_sentiment,score_x,id_x,popularity
0,2012-02-10,Tesla Model X video,it,[],[],https://twitter.com/elonmusk/status/1680222380...,False,16,12,47,[],1,0.00,0.00,0,0,1,0.752232,12121,75
1,2013-05-01,realbhuwan teslamotors Cool,en,[],[],https://twitter.com/elonmusk/status/3296526685...,False,5,1,1,[],1,0.65,0.35,1,1,1,0.637080,11733,7
2,2013-05-04,TeslaTrip you're welcome,en,[],[],https://twitter.com/elonmusk/status/3305718113...,False,2,0,2,[],1,0.90,0.80,1,1,1,0.554013,11713,4
3,2013-11-19,Mission of Tesla,en,[],[],https://twitter.com/elonmusk/status/4028037326...,False,434,139,672,[],1,0.00,0.00,0,0,1,0.761174,11526,1245
4,2014-04-01,Announcing a revolutionary Tesla driving techn...,en,[],[],https://twitter.com/elonmusk/status/4510947064...,False,604,120,652,[],1,0.00,0.00,0,1,1,0.540329,11440,1376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,2021-03-19,teslaownersSV,und,[],[],https://twitter.com/elonmusk/status/1372792170...,False,3143,189,128,"[{'screen_name': 'teslaownersSV', 'name': 'Tes...",1,0.00,0.00,0,0,1,0.907871,5524,3460
793,2021-03-19,teslaownersSV,und,[],[],https://twitter.com/elonmusk/status/1372792170...,False,3143,189,128,"[{'screen_name': 'teslaownersSV', 'name': 'Tes...",1,0.00,0.00,0,0,1,0.907871,25,3460
794,2021-03-19,teslaownersSV,und,[],[],https://twitter.com/elonmusk/status/1372792170...,False,3143,189,128,"[{'screen_name': 'teslaownersSV', 'name': 'Tes...",1,0.00,0.00,0,0,1,0.907871,25,3460
795,2021-03-19,teslaownersSV,und,[],[],https://twitter.com/elonmusk/status/1372792170...,False,3143,189,128,"[{'screen_name': 'teslaownersSV', 'name': 'Tes...",1,0.00,0.00,0,0,1,0.907871,5524,3460


In [216]:
joined = pd.merge(tweets_daily, stocks, on = "date", how = 'inner')
joined

,date,retweet,nlikes,nreplies,nretweets,Cluster,subjectivity,polarity,sentiment,bertwitter_sentiment,hf_sentiment,score_x,id_x,popularity,open,high,low,close,adj
0,2012-02-10,False,16,12,47,1,0.00,0.00,0,0,1,0.752232,12121,75,6.452000,6.454000,5.968000,6.220000,6.220000
1,2013-05-01,False,5,1,1,1,0.65,0.35,1,1,1,0.637080,11733,7,11.198000,11.198000,10.600000,10.656000,10.656000
2,2013-05-04,False,2,0,2,1,0.90,0.80,1,1,1,0.554013,11713,4,11.286000,11.613000,11.000000,11.405000,11.405000
3,2013-11-19,False,434,139,672,1,0.00,0.00,0,0,1,0.761174,11526,1245,23.886000,25.799999,23.844000,25.218000,25.218000
4,2014-04-01,False,604,120,652,1,0.00,0.00,0,1,1,0.540329,11440,1376,41.804001,43.632000,41.716000,43.394001,43.394001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2021-02-24,False,3891,219,265,1,1.00,0.00,0,0,-1,0.696392,185,4375,711.849976,745.000000,694.169983,742.020020,742.020020
274,2021-02-26,False,258579,15546,13309,1,0.00,0.00,0,0,1,0.437373,157,287434,700.000000,706.700012,659.510010,675.500000,675.500000
275,2021-03-01,False,3679,138,173,1,0.00,0.00,0,1,1,0.596494,152,3990,690.109985,719.000000,685.049988,718.429993,718.429993
276,2021-03-02,False,3107,182,99,1,0.00,0.00,0,0,1,0.925330,144,3388,718.280029,721.109985,685.000000,686.440002,686.440002


In [217]:
from scipy.stats import pearsonr, kendalltau, spearmanr
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [218]:
import scipy.stats

def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = scipy.stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [219]:
percents = [0,0.5,1,1.5,2]
types = ['sentiment', 'bertwitter_sentiment', 'hf_sentiment']
# corr_coef = pd.DataFrame(columns = ['Cluster','Type', 'Cramer','Pearson', 'Spearman', 'KendallTau' , 'percent_change'])
corr_coef = pd.DataFrame(columns = ['Cluster','Type', 'Cramer', 'percent_change'])
clusters = [1]
for c in clusters:
  clusterized = joined.loc[joined.Cluster == c].reset_index()
  for t in types:
    if t == 'hf_sentiment':
      df = stocks_changer(clusterized, 0)
      # pearson = round(pearsonr(df[t], df.change)[0], 4)*100
      # spearman = round(spearmanr(df[t], df.change)[0], 4)*100
      # kendall = round(kendalltau(df[t], df.change)[0], 4)*100
      confusion_matrix = pd.crosstab(df[t], df.change)
      # print(confusion_matrix)
      cramer = round(cramers_corrected_stat(confusion_matrix), 5)
      # corr_coef = corr_coef.append(pd.Series([c, t, cramer, pearson, spearman, kendall, 0], index = corr_coef.columns), ignore_index=True)
      corr_coef = corr_coef.append(pd.Series([c, t, cramer, 0], index = corr_coef.columns), ignore_index=True)
    else:
      for p in percents:
        df = stocks_changer(clusterized, p)
        # X, y = df[['nlikes','nreplies','sentiment']], df.open_dir
        # X_tr, X_te, y_tr, y_te = train_test_split(X,y, test_size = 0.2)
        # model = svm.SVC()
        # model.fit(X_tr,y_tr)
        # y_pr = model.predict(X_te)
        # accuracy = accuracy_score(y_te, y_pr)
        pearson = round(pearsonr(df[t], df.change)[0], 4)*100
        spearman = round(spearmanr(df[t], df.change)[0], 4)*100
        kendall = round(kendalltau(df[t], df.change)[0], 4)*100
        confusion_matrix = pd.crosstab(df[t], df.change)
        # print(confusion_matrix)
        cramer = round(cramers_corrected_stat(confusion_matrix), 5)
        # corr_coef = corr_coef.append(pd.Series([c, t, cramer, pearson, spearman, kendall, p], index = corr_coef.columns), ignore_index=True)
        corr_coef = corr_coef.append(pd.Series([c, t, cramer, p], index = corr_coef.columns), ignore_index=True)

In [220]:
corr_coef.to_csv("results.csv")
corr_coef

,Cluster,Type,Cramer,percent_change
0,1,sentiment,0.00000,0
1,1,sentiment,0.00000,0.5
2,1,sentiment,0.00000,1
3,1,sentiment,0.00000,1.5
4,1,sentiment,0.05627,2
5,1,bertwitter_sentiment,0.02713,0
6,1,bertwitter_sentiment,0.08000,0.5
7,1,bertwitter_sentiment,0.06913,1
8,1,bertwitter_sentiment,0.06898,1.5
9,1,bertwitter_sentiment,0.03616,2


In [221]:
pivoted = corr_coef.pivot(index = 'percent_change', columns = ['Type', 'Cluster'], values = 'Cramer').reset_index()
pivoted

Type,percent_change,sentiment,bertwitter_sentiment,hf_sentiment
Cluster,,1,1,1
0,0.0,0.00000,0.02713,0.02442
1,0.5,0.00000,0.08000,NaN
2,1.0,0.00000,0.06913,NaN
3,1.5,0.00000,0.06898,NaN
4,2.0,0.05627,0.03616,NaN


In [222]:
joined2 = pd.merge(tweets_daily, stocks, on = "date", how = 'inner')
cut = pd.to_datetime('2018-01-01')
joined2 = joined2.loc[joined2.date >= cut]
joined2

,date,retweet,nlikes,nreplies,nretweets,Cluster,subjectivity,polarity,sentiment,bertwitter_sentiment,hf_sentiment,score_x,id_x,popularity,open,high,low,close,adj
28,2018-01-29,False,405,9,6,1,0.0,0.0,0,0,1,0.700952,9018,420,67.970001,70.169998,67.655998,69.905998,69.905998
29,2018-03-04,False,7689,95,193,1,0.0,0.0,0,0,-1,0.739500,8914,7977,66.207499,67.423502,65.542002,66.758499,66.758499
30,2018-04-08,False,743,43,32,1,0.4,0.0,0,0,-1,0.735599,8772,818,60.105501,61.889001,58.156499,58.414000,58.414000
31,2018-05-01,False,4797,120,198,1,0.5,0.5,1,0,1,0.837018,8727,5115,58.702000,60.164001,58.644001,59.984001,59.984001
32,2018-05-24,False,2226,81,90,1,0.0,0.0,0,0,1,0.637870,8501,2397,55.680000,56.222000,54.978001,55.570000,55.570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2021-02-24,False,3891,219,265,1,1.0,0.0,0,0,-1,0.696392,185,4375,711.849976,745.000000,694.169983,742.020020,742.020020
274,2021-02-26,False,258579,15546,13309,1,0.0,0.0,0,0,1,0.437373,157,287434,700.000000,706.700012,659.510010,675.500000,675.500000
275,2021-03-01,False,3679,138,173,1,0.0,0.0,0,1,1,0.596494,152,3990,690.109985,719.000000,685.049988,718.429993,718.429993
276,2021-03-02,False,3107,182,99,1,0.0,0.0,0,0,1,0.925330,144,3388,718.280029,721.109985,685.000000,686.440002,686.440002


In [223]:
percents = [0,0.5,1,1.5,2]
types = ['sentiment', 'bertwitter_sentiment', 'hf_sentiment']
# corr_coef = pd.DataFrame(columns = ['Cluster','Type', 'Cramer','Pearson', 'Spearman', 'KendallTau' , 'percent_change'])
corr_coef = pd.DataFrame(columns = ['Cluster','Type', 'Cramer', 'percent_change'])
clusters = [1]
for c in clusters:
  clusterized = joined2.loc[joined2.Cluster == c].reset_index()
  for t in types:
    if t == 'hf_sentiment':
      df = stocks_changer(clusterized, 0)
      # pearson = round(pearsonr(df[t], df.change)[0], 4)*100
      # spearman = round(spearmanr(df[t], df.change)[0], 4)*100
      # kendall = round(kendalltau(df[t], df.change)[0], 4)*100
      confusion_matrix = pd.crosstab(df[t], df.change)
      # print(confusion_matrix)
      cramer = round(cramers_corrected_stat(confusion_matrix), 5)
      # corr_coef = corr_coef.append(pd.Series([c, t, cramer, pearson, spearman, kendall, 0], index = corr_coef.columns), ignore_index=True)
      corr_coef = corr_coef.append(pd.Series([c, t, cramer, 0], index = corr_coef.columns), ignore_index=True)
    else:
      for p in percents:
        df = stocks_changer(clusterized, p)
        # X, y = df[['nlikes','nreplies','sentiment']], df.open_dir
        # X_tr, X_te, y_tr, y_te = train_test_split(X,y, test_size = 0.2)
        # model = svm.SVC()
        # model.fit(X_tr,y_tr)
        # y_pr = model.predict(X_te)
        # accuracy = accuracy_score(y_te, y_pr)
        pearson = round(pearsonr(df[t], df.change)[0], 4)*100
        spearman = round(spearmanr(df[t], df.change)[0], 4)*100
        kendall = round(kendalltau(df[t], df.change)[0], 4)*100
        confusion_matrix = pd.crosstab(df[t], df.change)
        # print(confusion_matrix)
        cramer = round(cramers_corrected_stat(confusion_matrix), 5)
        # corr_coef = corr_coef.append(pd.Series([c, t, cramer, pearson, spearman, kendall, p], index = corr_coef.columns), ignore_index=True)
        corr_coef = corr_coef.append(pd.Series([c, t, cramer, p], index = corr_coef.columns), ignore_index=True)

In [224]:
corr_coef

,Cluster,Type,Cramer,percent_change
0,1,sentiment,0.00000,0
1,1,sentiment,0.01594,0.5
2,1,sentiment,0.07018,1
3,1,sentiment,0.03179,1.5
4,1,sentiment,0.09134,2
5,1,bertwitter_sentiment,0.00264,0
6,1,bertwitter_sentiment,0.09151,0.5
7,1,bertwitter_sentiment,0.05388,1
8,1,bertwitter_sentiment,0.05335,1.5
9,1,bertwitter_sentiment,0.00589,2


In [172]:
import pandas as pd
clusters = pd.read_csv('sentiment_twitters_numerical.csv')
clusters.head()

FileNotFoundError: ignored

In [ ]:
pd.set_option('display.max_rows', 15000)
pd.options.display.max_colwidth =300
clusters.loc[clusters.Cluster == -1].tweet.str.split(expand=True).stack().value_counts()

the                     648
to                      638
a                       551
is                      542
of                      475
in                      367
amp                     328
be                      283
for                     258
on                      216
I                       208
that                    199
but                     188
with                    178
will                    167
s                       163
it                      155
at                      153
are                     135
we                      132
by                      130
Model                   129
and                     120
not                     108
this                    107
Erdayastronaut          106
The                     105
.                       104
you                     102
from                    101
t                        98
about                    97
so                       95
It                       94
an                       92
have                